<a href="https://colab.research.google.com/github/sreeman-11021996/Fashion-Recommendation-system/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install import_ipynb
import import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import cv2
from keras.preprocessing.image import ImageDataGenerator
import math
from utils import clean_data_,map_label_dicts
import numpy as np
import os
import tensorflow

In [21]:
def train_val_split(df):
    # remove rows from the DataFrame which do not have corresponding images

    # shuffle the dataframe
    # sample(frac=1) -> gives us a shuffled dataset, then we reset the index
    df = df.sample(frac=1).reset_index(drop=True)

    # 90% for training and 10% for validation
    num_train_samples = math.floor(len(df) * 0.90)
    num_val_samples = math.floor(len(df) * 0.10)
    train_df = df[:num_train_samples].reset_index(drop=True)
    val_df = df[-num_val_samples:].reset_index(drop=True)
    return train_df, val_df

In [22]:
class FashionDataset:
    def __init__(self,df,labels=["gender","masterCategory","subCategory"]):
        df = clean_data_(df)
        gender,master,sub = map_label_dicts(df)
        self.labels = labels
        self.label_mappings(df)

        self.num_list_gender = gender
        self.num_list_master = master
        self.num_list_sub = sub
        
        # the training transforms and augmentations
        self.datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True,vertical_flip=True,
                                          rotation_range = 40,shear_range = 0.2,zoom_range = 0.2,
                                          width_shift_range=0.2,height_shift_range = 0.2)
       
    def label_mappings(self,df):
        df["classes"] = df[self.labels].apply(tuple, axis=1)
        self.df = df.reset_index()[["id","classes","index"]]
          

    def __len__(self):
        return len(self.df)

    def preprocess(self,img_path):
        image = image.load_img(img_path,target_size=(224,224))
        img_array = image.img_to_array(image)
        expanded_img_array = np.expand_dims(img_array, axis=0)
        preprocessed_img = expanded_img_array/255
        return preprocessed_img

    def __getitem__(self, index):
        # one image
        root_dir = "images"
        img_filename = str(self.df['id'][index]) + ".jpg"
        img_path = os.path.join(root_dir,img_filename)

        image = self.preprocess(img_path)

        cat_gender = self.df['gender'][index]
        label_gender = self.num_list_gender[cat_gender]
        cat_master = self.df['masterCategory'][index]
        label_master = self.num_list_master[cat_master]
        cat_sub = self.df['subCategory'][index]
        label_sub = self.num_list_sub[cat_sub]
        
        # image to float32 tensor
        image = tensorflow.convert_to_tensor(image, dtype=tensorflow.float32)
        # labels to long tensors
        label_gender = tensorflow.convert_to_tensor(label_gender, 
                                                    dtype=tensorflow.long)
        label_master = tensorflow.convert_to_tensor(label_master, 
                                                    dtype=tensorflow.long)
        label_sub = tensorflow.convert_to_tensor(label_sub, 
                                                 dtype=tensorflow.long)
        return {
            'image': image,
            'gender': label_gender,
            'master': label_master,
            'sub': label_sub
        }